<a href="https://colab.research.google.com/github/amit8507/X-tra-Telegram/blob/master/5_minute_strategy_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=06812627118cf0dec52c3c483a4476ab33a06d09b8c6444ddd4facd38e77e90b
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


In [ ]:
pip install backtesting

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173912 sha256=549ec3690b08ac6653d62d1d21c7578c634b7164ebf1dc64a235d34f305eb09e
  Stored in directory: /root/.cache/pip/wheels/e2/30/7f/19cbe31987c6ebdb47f1f510343249066711609e3da2d57176
Successfully built backtesting


In [ ]:
pip install bokeh==2.4.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 35.0 MB/s eta 0:00:00
  Attempting uninstall: bokeh
    Found existing installation: bokeh 3.6.1
    Uninstalling bokeh-3.6.1:
      Successfully uninstalled bokeh-3.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
holoviews 1.20.0 requires bokeh>=3.1, but you have bokeh 2.4.3 which is incompatible.
panel 1.5.4 requires bokeh<3.7.0,>=3.5.0, but you have bokeh 2.4.3 which is incompatible.


In [ ]:
import requests
import pandas as pd
import ta
from backtesting import Backtest, Strategy

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


In [ ]:
API_KEY = 'o0Kw2AoqnDNbCO3kZmGz51hMSAOVnagQ'

def get_xauusd_data(api_key, multiplier=5, timespan="minute", from_date="2024-05-15", to_date="2024-11-15"):
    url = f"https://api.polygon.io/v2/aggs/ticker/C:XAUUSD/range/{multiplier}/{timespan}/{from_date}/{to_date}"

    params = {
        "apiKey": 'o0Kw2AoqnDNbCO3kZmGz51hMSAOVnagQ',
        "limit": 100000
    }

    all_data = []
    next_url = url

    while next_url:
        response = requests.get(next_url, params=params)

        if response.status_code == 200:
            data = response.json()
            all_data.extend(data["results"])

            next_url = data.get("next_url")
        else:
            print(f"Error: {response.status_code}, {response.text}")
            break

    if all_data:
        df = pd.DataFrame(all_data)
        df['timestamp'] = pd.to_datetime(df['t'], unit='ms')
        return df[['timestamp', 'o', 'h', 'l', 'c', 'v']]
    else:
        return None

xauusd_data_5min = get_xauusd_data(API_KEY)
if xauusd_data_5min is not None:
    print(xauusd_data_5min.head())

            timestamp        o        h        l        c   v
0 2024-05-15 00:00:00  2355.94  2356.06  2355.49  2356.06  13
1 2024-05-15 00:05:00  2356.42  2356.42  2355.94  2355.94  13
2 2024-05-15 00:10:00  2355.69  2356.07  2355.58  2355.72  13
3 2024-05-15 00:15:00  2355.83  2356.40  2355.72  2355.97  14
4 2024-05-15 00:20:00  2355.94  2356.18  2355.86  2356.13  13


In [ ]:
df = xauusd_data_5min.rename(columns={'timestamp': 'Time', 'o': 'Open', 'h': 'High', 'l': 'Low', 'c': 'Close', 'v' : 'Volume'})
df

,Time,Open,High,Low,Close,Volume
0,2024-05-15 00:00:00,2355.94,2356.06,2355.49,2356.06,13
1,2024-05-15 00:05:00,2356.42,2356.42,2355.94,2355.94,13
2,2024-05-15 00:10:00,2355.69,2356.07,2355.58,2355.72,13
3,2024-05-15 00:15:00,2355.83,2356.40,2355.72,2355.97,14
4,2024-05-15 00:20:00,2355.94,2356.18,2355.86,2356.13,13
...,...,...,...,...,...,...
36327,2024-11-15 21:35:00,2561.48,2561.51,2561.20,2561.51,13
36328,2024-11-15 21:40:00,2561.41,2561.56,2561.12,2561.20,13
36329,2024-11-15 21:45:00,2561.05,2561.50,2561.05,2561.50,13
36330,2024-11-15 21:50:00,2561.39,2561.64,2561.15,2561.24,13


In [ ]:
df.set_index('Time', inplace = True)


df['Rsi'] = ta.momentum.RSIIndicator(df['Close'], window=14).rsi()

df['Ema9'] = ta.trend.EMAIndicator(df['Close'], window=9).ema_indicator()
df['Ema15'] = ta.trend.EMAIndicator(df['Close'], window=15).ema_indicator()
df['Ema21'] = ta.trend.EMAIndicator(df['Close'], window=21).ema_indicator()


df.dropna(inplace=True)

df

,Open,High,Low,Close,Volume,Rsi,Ema9,Ema15,Ema21
Time,,,,,,,,,
2024-05-15 01:40:00,2358.16,2358.36,2357.20,2357.21,14,58.082188,2356.753950,2356.470039,2356.333131
2024-05-15 01:45:00,2357.38,2357.42,2356.53,2356.56,13,52.312373,2356.715160,2356.481284,2356.353756
2024-05-15 01:50:00,2357.17,2358.26,2356.56,2356.98,13,55.395678,2356.768128,2356.543623,2356.410687
2024-05-15 01:55:00,2356.96,2357.90,2356.20,2357.49,14,58.872989,2356.912502,2356.661920,2356.508806
2024-05-15 02:00:00,2357.51,2357.89,2357.11,2357.89,13,61.413804,2357.108002,2356.815430,2356.634369
...,...,...,...,...,...,...,...,...,...
2024-11-15 21:35:00,2561.48,2561.51,2561.20,2561.51,13,41.977389,2561.857482,2562.063525,2562.294461
2024-11-15 21:40:00,2561.41,2561.56,2561.12,2561.20,13,40.028811,2561.725986,2561.955584,2562.194964
2024-11-15 21:45:00,2561.05,2561.50,2561.05,2561.50,13,42.796207,2561.680789,2561.898636,2562.131786


Optimizing strategy parameters for 6 month data.


---



for ema9

In [ ]:
class RSI_EMA_Strategy(Strategy):
    pip_size = 0.1
    percent_risk = 0.1


    def init(self):
        pass

    def next(self):
        if self.position:
            return

        equity_to_use = self._broker.equity
        lot_size = (equity_to_use * self.percent_risk) / (self.pip_size * self.sl_pips)
        units = int(lot_size)

        if units <= 0:
            return

        position_size = units

        if (self.data.Close[-1] > self.data.Open[-1] and
            self.data.Rsi[-1] < self.rsi_low and
            self.data.Close[-1] < self.data.Ema9[-1] - self.pip_difference * self.pip_size):

            entry_price = self.data.Close[-1]
            stop_loss = entry_price - self.sl_pips * self.pip_size
            take_profit = entry_price + self.target_pips * self.pip_size

            if stop_loss < entry_price < take_profit:
                self.buy(size=position_size, sl=stop_loss, tp=take_profit)

        elif (self.data.Close[-1] < self.data.Open[-1] and
              self.data.Rsi[-1] > self.rsi_high and
              self.data.Close[-1] > self.data.Ema9[-1] + self.pip_difference * self.pip_size):

            entry_price = self.data.Close[-1]
            stop_loss = entry_price + self.sl_pips * self.pip_size
            take_profit = entry_price - self.target_pips * self.pip_size

            if take_profit < entry_price < stop_loss:
                self.sell(size=position_size, sl=stop_loss, tp=take_profit)

target_pips_list = [15]
rsi_low_list = [20,30]
rsi_high_list = [60,70,80]
sl_pips_list = [15]
pip_difference_list = [10,15]

results = []

for target_pips in target_pips_list:
    for rsi_low in rsi_low_list:
        for rsi_high in rsi_high_list:
            for sl_pips in sl_pips_list:
                for pip_difference in pip_difference_list:

                    RSI_EMA_Strategy.target_pips = target_pips
                    RSI_EMA_Strategy.sl_pips = sl_pips
                    RSI_EMA_Strategy.rsi_low = rsi_low
                    RSI_EMA_Strategy.rsi_high = rsi_high
                    RSI_EMA_Strategy.pip_difference = pip_difference

                    bt = Backtest(df, RSI_EMA_Strategy, cash=10000, commission=0, margin=0.005)
                    stats = bt.run()

                    results.append({
                        'target_pips': target_pips,
                        'sl_pips': sl_pips,
                        'rsi_low': rsi_low,
                        'rsi_high': rsi_high,
                        'pip_difference': pip_difference,
                        'Final Equity': stats['Equity Final [$]'],
                        '# Trades': stats['# Trades'],
                        'Win Rate [%]': stats['Win Rate [%]'],
                        'Sharpe Ratio': stats['Sharpe Ratio'],
                        'Return': stats['Return [%]'],
                        'Max. Drawdown [%]': stats['Max. Drawdown [%]'],
                        'Avg. Drawdown [%]': stats['Avg. Drawdown [%]'],
                        'Worst Trade [%]': stats['Worst Trade [%]'],
                        'Best Trade [%]': stats['Best Trade [%]']
                    })

results_df = pd.DataFrame(results)


try:
    best_result = results_df.loc[results_df['Final Equity'].idxmax()]

    bt = Backtest(df, RSI_EMA_Strategy, cash=10000, commission=0, margin= 0.005)
    bt.run(target_pips=best_result['target_pips'], rsi_low=best_result['rsi_low'], rsi_high=best_result['rsi_high'], pip_difference=best_result['pip_difference'],sl_pips = best_result['sl_pips'])
    bt.plot()
except Exception as e:
    print(f"Error during plotting: {e}")

Error during plotting: Index.get_loc() got an unexpected keyword argument 'method'


/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:122: UserWarning: Data contains too many candlesticks to plot; downsampling to '30T'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:126: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df = df.resample(freq, label='right').agg(OHLCV_AGG).dropna()
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:136: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  equity_data = equity_data.resample(freq, label='right').agg(_EQUITY_AGG).dropna(how='all')
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:153: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  trades = trades.assign(count=1).resample(freq, on='Exit

In [ ]:
best_result

,2
target_pips,15.000000
sl_pips,15.000000
rsi_low,20.000000
rsi_high,70.000000
pip_difference,10.000000
Final Equity,73944.300000
# Trades,159.000000
Win Rate [%],54.716981
Sharpe Ratio,0.365573
Return,639.443000


for ema15

In [ ]:
class RSI_EMA_Strategy(Strategy):
    pip_size = 0.1
    percent_risk = 0.1


    def init(self):
        pass

    def next(self):
        if self.position:
            return

        equity_to_use = self._broker.equity
        lot_size = (equity_to_use * self.percent_risk) / (self.pip_size * self.sl_pips)
        units = int(lot_size)

        if units <= 0:
            return

        position_size = units

        if (self.data.Close[-1] > self.data.Open[-1] and
            self.data.Close[-2] > self.data.Open[-2] and
            self.data.Rsi[-2] < self.rsi_low and
            self.data.Close[-2] < self.data.Ema15[-2] - self.pip_difference * self.pip_size):

            entry_price = self.data.Close[-1]
            stop_loss = entry_price - self.sl_pips * self.pip_size
            take_profit = entry_price + self.target_pips * self.pip_size

            if stop_loss < entry_price < take_profit:
                self.buy(size=position_size, sl=stop_loss, tp=take_profit)

        elif (self.data.Close[-1] < self.data.Open[-1] and
              self.data.Close[-2] < self.data.Open[-2] and
              self.data.Rsi[-2] > self.rsi_high and
              self.data.Close[-2] > self.data.Ema15[-2] + self.pip_difference * self.pip_size):

            entry_price = self.data.Close[-1]
            stop_loss = entry_price + self.sl_pips * self.pip_size
            take_profit = entry_price - self.target_pips * self.pip_size

            if take_profit < entry_price < stop_loss:
                self.sell(size=position_size, sl=stop_loss, tp=take_profit)

target_pips_list = [15]
rsi_low_list = [20]
rsi_high_list = [60]
sl_pips_list = [15]
pip_difference_list = [10]

results = []

for target_pips in target_pips_list:
    for rsi_low in rsi_low_list:
        for rsi_high in rsi_high_list:
            for sl_pips in sl_pips_list:
                for pip_difference in pip_difference_list:

                    RSI_EMA_Strategy.target_pips = target_pips
                    RSI_EMA_Strategy.sl_pips = sl_pips
                    RSI_EMA_Strategy.rsi_low = rsi_low
                    RSI_EMA_Strategy.rsi_high = rsi_high
                    RSI_EMA_Strategy.pip_difference = pip_difference

                    bt = Backtest(df, RSI_EMA_Strategy, cash=10000, commission=0, margin=0.005)
                    stats = bt.run()

                    results.append({
                        'target_pips': target_pips,
                        'sl_pips': sl_pips,
                        'rsi_low': rsi_low,
                        'rsi_high': rsi_high,
                        'pip_difference': pip_difference,
                        'Final Equity': stats['Equity Final [$]'],
                        '# Trades': stats['# Trades'],
                        'Win Rate [%]': stats['Win Rate [%]'],
                        'Sharpe Ratio': stats['Sharpe Ratio'],
                        'Return': stats['Return [%]'],
                        'Max. Drawdown [%]': stats['Max. Drawdown [%]'],
                        'Avg. Drawdown [%]': stats['Avg. Drawdown [%]'],
                        'Worst Trade [%]': stats['Worst Trade [%]'],
                        'Best Trade [%]': stats['Best Trade [%]']
                    })

results_df = pd.DataFrame(results)


try:
    best_result = results_df.loc[results_df['Final Equity'].idxmax()]

    bt = Backtest(df, RSI_EMA_Strategy, cash=10000, commission=0, margin= 0.005)
    bt.run(target_pips=best_result['target_pips'], rsi_low=best_result['rsi_low'], rsi_high=best_result['rsi_high'], pip_difference=best_result['pip_difference'],sl_pips = best_result['sl_pips'])
    bt.plot()
except Exception as e:
    print(f"Error during plotting: {e}")

Error during plotting: Index.get_loc() got an unexpected keyword argument 'method'


/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:122: UserWarning: Data contains too many candlesticks to plot; downsampling to '30T'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:126: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df = df.resample(freq, label='right').agg(OHLCV_AGG).dropna()
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:136: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  equity_data = equity_data.resample(freq, label='right').agg(_EQUITY_AGG).dropna(how='all')
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:153: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  trades = trades.assign(count=1).resample(freq, on='Exit

In [ ]:
best_result

,0
target_pips,15.000000
sl_pips,15.000000
rsi_low,20.000000
rsi_high,60.000000
pip_difference,10.000000
Final Equity,404.330000
# Trades,803.000000
Win Rate [%],49.190535
Sharpe Ratio,0.000000
Return,-95.956700


for ema21

In [ ]:
class RSI_EMA_Strategy(Strategy):
    pip_size = 0.1
    percent_risk = 0.1


    def init(self):
        pass

    def next(self):
        if self.position:
            return

        equity_to_use = self._broker.equity
        lot_size = (equity_to_use * self.percent_risk) / (self.pip_size * self.sl_pips)
        units = int(lot_size)

        if units <= 0:
            return

        position_size = units

        if (self.data.Close[-1] > self.data.Open[-1] and
            self.data.Close[-2] > self.data.Open[-2] and
            self.data.Rsi[-2] < self.rsi_low and
            self.data.Close[-2] < self.data.Ema21[-2] - self.pip_difference * self.pip_size):

            entry_price = self.data.Close[-1]
            stop_loss = entry_price - self.sl_pips * self.pip_size
            take_profit = entry_price + self.target_pips * self.pip_size

            if stop_loss < entry_price < take_profit:
                self.buy(size=position_size, sl=stop_loss, tp=take_profit)

        elif (self.data.Close[-1] < self.data.Open[-1] and
              self.data.Close[-2] < self.data.Open[-2] and
              self.data.Rsi[-2] > self.rsi_high and
              self.data.Close[-2] > self.data.Ema21[-2] + self.pip_difference * self.pip_size):

            entry_price = self.data.Close[-1]
            stop_loss = entry_price + self.sl_pips * self.pip_size
            take_profit = entry_price - self.target_pips * self.pip_size

            if take_profit < entry_price < stop_loss:
                self.sell(size=position_size, sl=stop_loss, tp=take_profit)

target_pips_list = [15]
rsi_low_list = [20]
rsi_high_list = [60]
sl_pips_list = [15]
pip_difference_list = [10]

results = []

for target_pips in target_pips_list:
    for rsi_low in rsi_low_list:
        for rsi_high in rsi_high_list:
            for sl_pips in sl_pips_list:
                for pip_difference in pip_difference_list:

                    RSI_EMA_Strategy.target_pips = target_pips
                    RSI_EMA_Strategy.sl_pips = sl_pips
                    RSI_EMA_Strategy.rsi_low = rsi_low
                    RSI_EMA_Strategy.rsi_high = rsi_high
                    RSI_EMA_Strategy.pip_difference = pip_difference

                    bt = Backtest(df, RSI_EMA_Strategy, cash=10000, commission=0, margin=0.005)
                    stats = bt.run()

                    results.append({
                        'target_pips': target_pips,
                        'sl_pips': sl_pips,
                        'rsi_low': rsi_low,
                        'rsi_high': rsi_high,
                        'pip_difference': pip_difference,
                        'Final Equity': stats['Equity Final [$]'],
                        '# Trades': stats['# Trades'],
                        'Win Rate [%]': stats['Win Rate [%]'],
                        'Sharpe Ratio': stats['Sharpe Ratio'],
                        'Return': stats['Return [%]'],
                        'Max. Drawdown [%]': stats['Max. Drawdown [%]'],
                        'Avg. Drawdown [%]': stats['Avg. Drawdown [%]'],
                        'Worst Trade [%]': stats['Worst Trade [%]'],
                        'Best Trade [%]': stats['Best Trade [%]']
                    })

results_df = pd.DataFrame(results)


try:
    best_result = results_df.loc[results_df['Final Equity'].idxmax()]

    bt = Backtest(df, RSI_EMA_Strategy, cash=10000, commission=0, margin= 0.005)
    bt.run(target_pips=best_result['target_pips'], rsi_low=best_result['rsi_low'], rsi_high=best_result['rsi_high'], pip_difference=best_result['pip_difference'],sl_pips = best_result['sl_pips'])
    bt.plot()
except Exception as e:
    print(f"Error during plotting: {e}")

Error during plotting: Index.get_loc() got an unexpected keyword argument 'method'


/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:122: UserWarning: Data contains too many candlesticks to plot; downsampling to '30T'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:126: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df = df.resample(freq, label='right').agg(OHLCV_AGG).dropna()
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:136: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  equity_data = equity_data.resample(freq, label='right').agg(_EQUITY_AGG).dropna(how='all')
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:153: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  trades = trades.assign(count=1).resample(freq, on='Exit

In [ ]:
best_result

,0
target_pips,15.000000
sl_pips,15.000000
rsi_low,20.000000
rsi_high,60.000000
pip_difference,10.000000
Final Equity,174.690000
# Trades,882.000000
Win Rate [%],48.866213
Sharpe Ratio,0.000000
Return,-98.253100


so we get best parameter as


*  rsi_low = 20
*  rsi_high = 70
*  pip_difference = 10
*  ema9





**Backtesting for 1 month data for best parameters**

In [ ]:
API_KEY = 'o0Kw2AoqnDNbCO3kZmGz51hMSAOVnagQ'

def get_xauusd_data(api_key, multiplier=5, timespan="minute", from_date="2024-10-15", to_date="2024-11-15"):
    url = f"https://api.polygon.io/v2/aggs/ticker/C:XAUUSD/range/{multiplier}/{timespan}/{from_date}/{to_date}"

    params = {
        "apiKey": 'o0Kw2AoqnDNbCO3kZmGz51hMSAOVnagQ',
        "limit": 100000
    }

    all_data = []
    next_url = url

    while next_url:
        response = requests.get(next_url, params=params)

        if response.status_code == 200:
            data = response.json()
            all_data.extend(data["results"])

            next_url = data.get("next_url")
        else:
            print(f"Error: {response.status_code}, {response.text}")
            break

    if all_data:
        df = pd.DataFrame(all_data)
        df['timestamp'] = pd.to_datetime(df['t'], unit='ms')
        return df[['timestamp', 'o', 'h', 'l', 'c', 'v']]
    else:
        return None

xauusd_data_5min = get_xauusd_data(API_KEY)
if xauusd_data_5min is not None:
    print(xauusd_data_5min.head())

            timestamp        o        h        l        c   v
0 2024-10-15 00:00:00  2649.03  2649.92  2649.03  2649.63  13
1 2024-10-15 00:05:00  2649.43  2649.93  2648.95  2649.24  13
2 2024-10-15 00:10:00  2649.47  2649.47  2648.45  2648.45  13
3 2024-10-15 00:15:00  2648.56  2648.56  2647.93  2648.22  13
4 2024-10-15 00:20:00  2648.65  2648.83  2647.60  2648.73  14


In [ ]:
df = xauusd_data_5min.rename(columns={'timestamp': 'Time', 'o': 'Open', 'h': 'High', 'l': 'Low', 'c': 'Close', 'v' : 'Volume'})

In [ ]:
df.set_index('Time', inplace = True)


df['Rsi'] = ta.momentum.RSIIndicator(df['Close'], window=14).rsi()

df['Ema9'] = ta.trend.EMAIndicator(df['Close'], window=9).ema_indicator()
df['Ema15'] = ta.trend.EMAIndicator(df['Close'], window=15).ema_indicator()
df['Ema21'] = ta.trend.EMAIndicator(df['Close'], window=21).ema_indicator()


df.dropna(inplace=True)


In [ ]:
class RSI_EMA_Strategy(Strategy):
    pip_size = 0.1
    percent_risk = 0.1


    def init(self):
        pass

    def next(self):
        if self.position:
            return

        equity_to_use = self._broker.equity
        lot_size = (equity_to_use * self.percent_risk) / (self.pip_size * self.sl_pips)
        units = int(lot_size)

        if units <= 0:
            return

        position_size = units

        if (self.data.Close[-1] > self.data.Open[-1] and
            self.data.Close[-2] > self.data.Open[-2] and
            self.data.Rsi[-2] < self.rsi_low and
            self.data.Close[-2] < self.data.Ema9[-2] - self.pip_difference * self.pip_size):

            entry_price = self.data.Close[-1]
            stop_loss = entry_price - self.sl_pips * self.pip_size
            take_profit = entry_price + self.target_pips * self.pip_size

            if stop_loss < entry_price < take_profit:
                self.buy(size=position_size, sl=stop_loss, tp=take_profit)

        elif (self.data.Close[-1] < self.data.Open[-1] and
              self.data.Close[-2] < self.data.Open[-2] and
              self.data.Rsi[-2] > self.rsi_high and
              self.data.Close[-2] > self.data.Ema9[-2] + self.pip_difference * self.pip_size):

            entry_price = self.data.Close[-1]
            stop_loss = entry_price + self.sl_pips * self.pip_size
            take_profit = entry_price - self.target_pips * self.pip_size

            if take_profit < entry_price < stop_loss:
                self.sell(size=position_size, sl=stop_loss, tp=take_profit)

target_pips_list = [15]
rsi_low_list = [20]
rsi_high_list = [70]
sl_pips_list = [15]
pip_difference_list = [10]

results = []

for target_pips in target_pips_list:
    for rsi_low in rsi_low_list:
        for rsi_high in rsi_high_list:
            for sl_pips in sl_pips_list:
                for pip_difference in pip_difference_list:

                    RSI_EMA_Strategy.target_pips = target_pips
                    RSI_EMA_Strategy.sl_pips = sl_pips
                    RSI_EMA_Strategy.rsi_low = rsi_low
                    RSI_EMA_Strategy.rsi_high = rsi_high
                    RSI_EMA_Strategy.pip_difference = pip_difference

                    bt = Backtest(df, RSI_EMA_Strategy, cash=10000, commission=0, margin=0.005)
                    stats = bt.run()

                    results.append({
                        'target_pips': target_pips,
                        'sl_pips': sl_pips,
                        'rsi_low': rsi_low,
                        'rsi_high': rsi_high,
                        'pip_difference': pip_difference,
                        'Final Equity': stats['Equity Final [$]'],
                        '# Trades': stats['# Trades'],
                        'Win Rate [%]': stats['Win Rate [%]'],
                        'Sharpe Ratio': stats['Sharpe Ratio'],
                        'Return': stats['Return [%]'],
                        'Max. Drawdown [%]': stats['Max. Drawdown [%]'],
                        'Avg. Drawdown [%]': stats['Avg. Drawdown [%]'],
                        'Worst Trade [%]': stats['Worst Trade [%]'],
                        'Best Trade [%]': stats['Best Trade [%]']
                    })

results_df = pd.DataFrame(results)


try:
    best_result = results_df.loc[results_df['Final Equity'].idxmax()]

    bt = Backtest(df, RSI_EMA_Strategy, cash=10000, commission=0, margin= 0.005)
    bt.run(target_pips=best_result['target_pips'], rsi_low=best_result['rsi_low'], rsi_high=best_result['rsi_high'], pip_difference=best_result['pip_difference'],sl_pips = best_result['sl_pips'])
    bt.plot()
except Exception as e:
    print(f"Error during plotting: {e}")

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:455: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df2 = (df.assign(_width=1).set_index('datetime')


In [ ]:
best_result

,0
target_pips,15.000000
sl_pips,15.000000
rsi_low,20.000000
rsi_high,70.000000
pip_difference,10.000000
Final Equity,36249.450000
# Trades,25.000000
Win Rate [%],68.000000
Sharpe Ratio,0.218608
Return,262.494500


Does it works for other pairs

In [ ]:
API_KEY = 'o0Kw2AoqnDNbCO3kZmGz51hMSAOVnagQ'

def get_eurusd_data(api_key, multiplier=5, timespan="minute", from_date="2024-10-15", to_date="2024-11-15"):
    url = f"https://api.polygon.io/v2/aggs/ticker/C:EURUSD/range/{multiplier}/{timespan}/{from_date}/{to_date}"

    params = {
        "apiKey": 'o0Kw2AoqnDNbCO3kZmGz51hMSAOVnagQ',
        "limit": 100000
    }

    all_data = []
    next_url = url

    while next_url:
        response = requests.get(next_url, params=params)

        if response.status_code == 200:
            data = response.json()
            all_data.extend(data["results"])

            next_url = data.get("next_url")
        else:
            print(f"Error: {response.status_code}, {response.text}")
            break

    if all_data:
        df = pd.DataFrame(all_data)
        df['timestamp'] = pd.to_datetime(df['t'], unit='ms')
        return df[['timestamp', 'o', 'h', 'l', 'c', 'v']]
    else:
        return None

eurusd_data_5min = get_eurusd_data(API_KEY)
if eurusd_data_5min is not None:
    print(eurusd_data_5min.head())

            timestamp        o        h       l        c    v
0 2024-10-15 00:00:00  1.09085  1.09096  1.0906  1.09075  449
1 2024-10-15 00:05:00  1.09075  1.09080  1.0904  1.09049  416
2 2024-10-15 00:10:00  1.09052  1.09060  1.0903  1.09041  342
3 2024-10-15 00:15:00  1.09042  1.09070  1.0903  1.09052  344
4 2024-10-15 00:20:00  1.09053  1.09080  1.0904  1.09062  411


In [ ]:
df = eurusd_data_5min.rename(columns={'timestamp': 'Time', 'o': 'Open', 'h': 'High', 'l': 'Low', 'c': 'Close', 'v' : 'Volume'})

In [ ]:
df.set_index('Time', inplace = True)


df['Rsi'] = ta.momentum.RSIIndicator(df['Close'], window=14).rsi()

df['Ema9'] = ta.trend.EMAIndicator(df['Close'], window=9).ema_indicator()
df['Ema15'] = ta.trend.EMAIndicator(df['Close'], window=15).ema_indicator()
df['Ema21'] = ta.trend.EMAIndicator(df['Close'], window=21).ema_indicator()


df.dropna(inplace=True)

In [ ]:
class RSI_EMA_Strategy(Strategy):
    pip_size = 0.0001
    percent_risk = 0.1


    def init(self):
        pass

    def next(self):
        if self.position:
            return

        equity_to_use = self._broker.equity
        lot_size = (equity_to_use * self.percent_risk) / (self.pip_size * self.sl_pips)
        units = int(lot_size)

        if units <= 0:
            return

        position_size = units

        if (self.data.Close[-1] > self.data.Open[-1] and
            self.data.Close[-2] > self.data.Open[-2] and
            self.data.Rsi[-2] < self.rsi_low and
            self.data.Close[-2] < self.data.Ema9[-2] - self.pip_difference * self.pip_size):

            entry_price = self.data.Close[-1]
            stop_loss = entry_price - self.sl_pips * self.pip_size
            take_profit = entry_price + self.target_pips * self.pip_size

            if stop_loss < entry_price < take_profit:
                self.buy(size=position_size, sl=stop_loss, tp=take_profit)

        elif (self.data.Close[-1] < self.data.Open[-1] and
              self.data.Close[-2] < self.data.Open[-2] and
              self.data.Rsi[-2] > self.rsi_high and
              self.data.Close[-2] > self.data.Ema9[-2] + self.pip_difference * self.pip_size):

            entry_price = self.data.Close[-1]
            stop_loss = entry_price + self.sl_pips * self.pip_size
            take_profit = entry_price - self.target_pips * self.pip_size

            if take_profit < entry_price < stop_loss:
                self.sell(size=position_size, sl=stop_loss, tp=take_profit)

target_pips_list = [15]
rsi_low_list = [30]
rsi_high_list = [60]
sl_pips_list = [15]
pip_difference_list = [10]

results = []

for target_pips in target_pips_list:
    for rsi_low in rsi_low_list:
        for rsi_high in rsi_high_list:
            for sl_pips in sl_pips_list:
                for pip_difference in pip_difference_list:

                    RSI_EMA_Strategy.target_pips = target_pips
                    RSI_EMA_Strategy.sl_pips = sl_pips
                    RSI_EMA_Strategy.rsi_low = rsi_low
                    RSI_EMA_Strategy.rsi_high = rsi_high
                    RSI_EMA_Strategy.pip_difference = pip_difference

                    bt = Backtest(df, RSI_EMA_Strategy, cash=10000, commission=0, margin=0.005)
                    stats = bt.run()

                    results.append({
                        'target_pips': target_pips,
                        'sl_pips': sl_pips,
                        'rsi_low': rsi_low,
                        'rsi_high': rsi_high,
                        'pip_difference': pip_difference,
                        'Final Equity': stats['Equity Final [$]'],
                        '# Trades': stats['# Trades'],
                        'Win Rate [%]': stats['Win Rate [%]'],
                        'Sharpe Ratio': stats['Sharpe Ratio'],
                        'Return': stats['Return [%]'],
                        'Max. Drawdown [%]': stats['Max. Drawdown [%]'],
                        'Avg. Drawdown [%]': stats['Avg. Drawdown [%]'],
                        'Worst Trade [%]': stats['Worst Trade [%]'],
                        'Best Trade [%]': stats['Best Trade [%]']
                    })

results_df = pd.DataFrame(results)


try:
    best_result = results_df.loc[results_df['Final Equity'].idxmax()]

    bt = Backtest(df, RSI_EMA_Strategy, cash=10000, commission=0, margin= 0.005)
    bt.run(target_pips=best_result['target_pips'], rsi_low=best_result['rsi_low'], rsi_high=best_result['rsi_high'], pip_difference=best_result['pip_difference'],sl_pips = best_result['sl_pips'])
    bt.plot()
except Exception as e:
    print(f"Error during plotting: {e}")

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:455: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df2 = (df.assign(_width=1).set_index('datetime')


As we can see it doesn't works.